# Analysis of the risk of bitcoin

## Setup

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal

In [ ]:
# Set charts theme
sns.set_theme(style="darkgrid", rc={"grid.alpha": 0.33})
plt.style.use("dark_background")

# Save chart as png function
def save_chart_as_png(filename: str) -> None:
    plt.savefig(
        f"../images/{filename}.png",
        format="png",
        dpi=300,
        orientation="landscape",
        bbox_inches="tight",
    )

In [ ]:
# Get all dfs
def get_df(csv_basename: str, value_column: str) -> pd.DataFrame:
    # Get df from CSV and set date as index
    df = pd.read_csv(f"../data/{csv_basename}.csv", index_col="date", parse_dates=True)
    
    # Get OHLC average as value
    df[value_column] = df[["open", "high", "low", "close"]].mean(axis=1)
    df.drop(columns=["open", "high", "low", "close"], inplace=True)

    return df

df_btc = get_df("BTC", "price")
df_sp500 = get_df("SP500", "price")
df_us10y = get_df("US10Y", "yield")
df_au = get_df("AU", "price")
df_wti = get_df("WTI", "price")
df_chf = get_df("CHF", "rate")

## Volatility of bitcoin across time (EWMA method)

In [ ]:
# Get log price change
df_btc["price_change_log"] = np.log(df_btc["price"] / df_btc["price"].shift(1))

In [ ]:
# Biggest price changes
df_btc.loc[df_btc["price_change_log"].abs().sort_values(ascending=False).head(10).index, ["price_change_log"]].T

In [ ]:
# Get 90-day and 1-year volatility with the exponentially weighted moving average (EWMA) method
df_btc["volatility_90d"] = df_btc["price_change_log"].ewm(span=90).std()
df_btc["volatility_1y"] = df_btc["price_change_log"].ewm(span=365).std()

In [ ]:
# Change first cycle values to NaN to avoid having unreliable volatility measures
df_btc.loc[:df_btc.index[89], "volatility_90d"] = np.nan
df_btc.loc[:df_btc.index[364], "volatility_1y"] = np.nan

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc, x=df_btc.index, y="volatility_90d", label="90-day volatility", color="#00f8ff", linewidth=0.75)
sns.lineplot(data=df_btc, x=df_btc.index, y="volatility_1y", label="1-year volatility", color="#ff5b00", linewidth=1)

plt.title("Volatility of the price of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

save_chart_as_png("2_BTC_vlt")

In [ ]:
# All-time high 1-year volatility
ath_date = df_btc["volatility_1y"].idxmax()
df_btc.loc[[ath_date], ["price", "volatility_1y"]]

In [ ]:
# All-time low 1-year volatility
atl_date = df_btc["volatility_1y"].idxmin()
df_btc.loc[[atl_date], ["price", "volatility_1y"]]

In [ ]:
# Top 5 peaks of 1-year volatility
peaks, _ = signal.find_peaks(df_btc["volatility_1y"], distance=500)
df_btc.iloc[peaks].nlargest(5, "volatility_1y").sort_values("date")[["volatility_1y"]].T

In [ ]:
# Top 5 valleys of 1-year volatility
valleys, _ = signal.find_peaks(-df_btc["volatility_1y"], distance=500)
df_btc.iloc[valleys].nsmallest(5, "volatility_1y").sort_values("date")[["volatility_1y"]].T

In [ ]:
# Average 1-year volatility
df_btc["volatility_1y"].mean().round(3)

**Key takeaways:**
- ...

### Comparison of volatility with other assets

In [ ]:
# Resample all dfs to weekly frequency, to be able to compare with bitcoin's 365 trading days
df_btc_w = df_btc[["price"]].resample("W").mean()
df_sp500_w = df_sp500.resample("W").mean()
df_us10y_w = df_us10y.resample("W").mean()
df_au_w = df_au.resample("W").mean()
df_wti_w = df_wti.resample("W").mean()
df_chf_w = df_chf.resample("W").mean()

In [ ]:
# Get 1-year volatility for each weekly df
for df in [df_btc_w, df_sp500_w, df_us10y_w, df_wti_w, df_au_w, df_chf_w]:
    # Replace non-positive values with NaN to avoid issues in log calculation (only the oil df has non-positive prices)
    df.iloc[:, 0] = df.iloc[:, 0].where(df.iloc[:, 0] > 0, np.nan)
    # Get log returns based on first column
    df["log_returns"] = np.log(df.iloc[:, 0] / df.iloc[:, 0].shift(1))
    # Get 1-year volatility with the exponentially weighted moving average (EWMA) method
    df["volatility_1y"] = df["log_returns"].ewm(span=52).std()
    # Remove first cycle to avoid having unreliable volatility measures
    df.drop(index=df.index[:51], inplace=True)

In [ ]:
plt.figure(figsize=(10, 6))

sns.lineplot(data=df_btc_w, x=df_btc_w.index, y="volatility_1y", label="Bitcoin", color="orange", linewidth=1)
sns.lineplot(data=df_sp500_w, x=df_sp500_w.index, y="volatility_1y", label="S&P 500", color="crimson", linewidth=1)
sns.lineplot(data=df_us10y_w, x=df_us10y_w.index, y="volatility_1y", label="US 10-year yield", color="dodgerblue", linewidth=1)
sns.lineplot(data=df_au_w, x=df_au_w.index, y="volatility_1y", label="Gold futures", color="gold", linewidth=1)
sns.lineplot(data=df_wti_w, x=df_wti_w.index, y="volatility_1y", label="Crude oil futures", color="limegreen", linewidth=1)
sns.lineplot(data=df_chf_w, x=df_chf_w.index, y="volatility_1y", label="USD/CHF", color="mediumorchid", linewidth=1)

plt.title("Comparison of the 1-year volatility of bitcoin with other assets over time")
plt.xlabel("")
plt.ylabel("")

#plt.yscale("log")
plt.ylim([0, 0.3])

save_chart_as_png("2_BTC_vlt_comparison")

In [ ]:
# Create table with average 1-year volatility of each asset for different timeframes
def get_timeframes_avg_vlt(df: pd.DataFrame) -> dict[str, float]:
    return {
        "Total avg volatility": round(df["volatility_1y"].mean(), 4),
        "2011-2016 avg": round(df[df.index.year < 2016]["volatility_1y"].mean(), 4),
        "2016-2020 avg": round(df[(df.index.year >= 2016) & (df.index.year < 2020)]["volatility_1y"].mean(), 4),
        "2020-2024 avg": round(df[df.index.year >= 2020]["volatility_1y"].mean(), 4),
    }

df_avg_vlt = pd.DataFrame({
    "Bitcoin": get_timeframes_avg_vlt(df_btc_w),
    "S&P 500": get_timeframes_avg_vlt(df_sp500_w),
    "US 10-year yield": get_timeframes_avg_vlt(df_us10y_w),
    "Gold futures": get_timeframes_avg_vlt(df_au_w),
    "Crude oil futures": get_timeframes_avg_vlt(df_wti_w),
    "USD/CHF": get_timeframes_avg_vlt(df_chf_w),
}).T

df_avg_vlt.sort_values(by=df_avg_vlt.columns[0], ascending=False)

**Key takeaways:**
- ...

## Yearly volatility across time

In [ ]:
# Group by year and get standard deviation of price change along with the number of days
df_btc_yearly = df_btc.groupby(df_btc.index.year).agg(
    volatility=("price_change_log", "std"),
    num_days=("price_change_log", "count"),
)

In [ ]:
# Annualize the volatility for incomplete years (2010 and 2024), multiplying by the square root of the division of 365 by number of days
df_btc_yearly.loc[df_btc_yearly["num_days"] < 365, "volatility"] *= (365 / df_btc_yearly["num_days"])**0.5

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=df_btc_yearly, x=df_btc_yearly.index, y="volatility", color="#f7931a")

plt.title("Yearly volatility of bitcoin across time")
plt.xlabel("")
plt.ylabel("")

save_chart_as_png("2_BTC_yearly_vlt")

In [ ]:
# Create table with yearly volatility stats
pd.DataFrame({
    "Average yearly volatility": [round(df_btc_yearly["volatility"].mean(), 4)],
    "Median yearly volatility": [round(df_btc_yearly["volatility"].median(), 4)],
    "Standard deviation": [round(df_btc_yearly["volatility"].std(), 4)],
    "Min yearly volatility": [round(df_btc_yearly["volatility"].min(), 4)],
    "Max yearly volatility": [round(df_btc_yearly["volatility"].max(), 4)],
})

**Key takeaways:**
- ...

## Volatility vs price (90-day rolling values)

In [ ]:
# Get 90-day moving average price
df_btc["price_90d_ma"] = df_btc["price"].rolling(window=90).mean()

In [ ]:
# Get 90-day rolling volatility with the standard deviation method
df_btc["volatility_90d_mstd"] = df_btc["price_change_log"].rolling(window=90).std()

In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(data=df_btc, x="price_90d_ma", y="volatility_90d_mstd", alpha=0.7, linewidth=0.2, color="yellow")

plt.title("Volatility vs price of bitcoin")
plt.xlabel("Average price (90-day window)")
plt.ylabel("Volatility (90-day window)")

plt.xscale("log")

plt.gca().xaxis.set_major_formatter(
    FuncFormatter(lambda x, _: f"{int(x)}" if x < 1000 and x.is_integer()
                  else (f"{x:.1f}" if x < 1 else f"{int(x/1000)}K"))
)
plt.xlim(0.05, 100_000)

save_chart_as_png("2_BTC_vlt_vs_price")

In [ ]:
# Pearson correlation coefficient betweent volatility and price
df_btc["price_90d_ma"].corr(df_btc["volatility_90d_mstd"]).round(2)

**Key takeaways:**
- ...

## Value at risk (VaR) ⚠️

### Historical method

In [ ]:
# Create a table with 95% and 99% confidence interval VaR for different time horizons
def calculate_hist_var(time_horizon_days: int, confidence_interval: float) -> float:
    # Aggregate log returns over the specified time horizon
    aggregated_returns = df_btc["price_change_log"].rolling(window=time_horizon_days).sum().dropna()

    # Convert confidence interval to corresponding percentile
    percentile = (1 - confidence_interval) * 100

    # Calculate the historical VaR as the value at the specified percentile of aggregated returns
    return -np.percentile(aggregated_returns, percentile).round(2)

confidence_intervals = [0.95, 0.99]

pd.DataFrame({
    "Confidence Interval": confidence_intervals,
    "1-month VaR": [calculate_hist_var(30, ci) for ci in confidence_intervals],
    "1-quarter VaR": [calculate_hist_var(90, ci) for ci in confidence_intervals],
    "1-year VaR": [calculate_hist_var(365, ci) for ci in confidence_intervals],
})

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 6), sharey=True)

time_horizons_days = [30, 90, 365]
colors = ["turquoise", "purple", "green"]
titles = ["30-day historical returns", "90-day historical returns", "365-day historical returns"]

for i, time_horizon_days in enumerate(time_horizons_days):
    aggregated_returns = df_btc["price_change_log"].rolling(window=time_horizon_days).sum().dropna()
    
    sns.histplot(aggregated_returns, stat="probability", binwidth=0.1, binrange=(-1, 1), color=colors[i], edgecolor="white", alpha=0.75, ax=axes[i])
    axes[i].axvline(np.percentile(aggregated_returns, 5), color="orange", linewidth=1.5, linestyle="--", label="VaR at 95% confidence level")
    axes[i].axvline(np.percentile(aggregated_returns, 1), color="red", linewidth=1.5, linestyle="--", label="VaR at 99% confidence level")
    
    axes[i].set_title(f"Distribution of the {titles[i]}")
    axes[i].set_xlabel("")
    
    axes[i].set_xlim(-1, 1)

axes[0].set_ylabel("Probability")
axes[2].legend(loc="upper right")

plt.tight_layout()

**Key takeaways:**
- ...

### Monte Carlo method

In [ ]:
# Create a table with 95% and 99% confidence interval VaR for different time horizons
def calculate_monte_carlo_var(mean: float, std: float, num_simulations: int, time_horizon_days: int, confidence_interval: float) -> float:
    # Simulate future returns using a normal distribution (output is array of x days by y simulations)
    simulated_returns = np.random.normal(mean, std, (num_simulations, time_horizon_days))
    
    # Aggregate returns over the time horizon (sum x days of each simulation)
    aggregated_returns = simulated_returns.sum(axis=1)

    # Convert confidence interval to corresponding percentile
    percentile = (1 - confidence_interval) * 100

    # Calculate VaR as the value at the specified percentile of simulated returns
    return -np.percentile(aggregated_returns, percentile).round(2)
    
mean = df_btc["price_change_log"].mean()
std = df_btc["price_change_log"].std()
num_simulations = 10_000
confidence_intervals = [0.95, 0.99]

pd.DataFrame({
    "Confidence Interval": confidence_intervals,
    "1-month VaR": [calculate_monte_carlo_var(mean, std, num_simulations, 30, ci) for ci in confidence_intervals],
    "1-quarter VaR": [calculate_monte_carlo_var(mean, std, num_simulations, 90, ci) for ci in confidence_intervals],
    "1-year VaR": [calculate_monte_carlo_var(mean, std, num_simulations, 365, ci) for ci in confidence_intervals],
})

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 6), sharey=True)

mean = df_btc["price_change_log"].mean()
std = df_btc["price_change_log"].std()
num_simulations = 10_000
time_horizons_days = [30, 90, 365]
colors = ["turquoise", "purple", "green"]
titles = ["30-day simulated returns", "90-day simulated returns", "365-day simulated returns"]

for i, time_horizon_days in enumerate(time_horizons_days):
    simulated_returns = np.random.normal(mean, std, (num_simulations, time_horizon_days))  
    aggregated_returns = simulated_returns.sum(axis=1)
    
    sns.histplot(aggregated_returns, stat="probability", binwidth=0.1, binrange=(-1, 1), color=colors[i], edgecolor="white", alpha=0.75, ax=axes[i])
    axes[i].axvline(np.percentile(aggregated_returns, 5), color="orange", linewidth=1.5, linestyle="--", label="VaR at 95% confidence level")
    axes[i].axvline(np.percentile(aggregated_returns, 1), color="red", linewidth=1.5, linestyle="--", label="VaR at 99% confidence level")
    
    axes[i].set_title(f"Distribution of the {titles[i]}")
    axes[i].set_xlabel("")
    
    axes[i].set_xlim(-1, 1)

axes[0].set_ylabel("Probability")
axes[2].legend(loc="upper right")

plt.tight_layout()

**Key takeaways:**
- ...